In [ ]:
import re
import pandas as pd
import numpy as np
import json
import subprocess
import glob
import os

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors
from matplotlib import cm
matplotlib.style.use('ggplot')
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch

In [ ]:
Mode = 'OneTest'

In [ ]:
Mode = 'Runs'

In [ ]:
if Mode='OneTest':
    eff_beg = 1.0
    eff_end = 1.0
    cost_beg = 1.0
    cost_end = 1.0
elif Mode = 'Runs':
    eff_beg = 1.0
    eff_end = 1.2
    cost_beg = 0.8
    cost_end = 1.2
    

In [ ]:
N_SEEDS = 150
N_SEI = 20
N_TESTS = 4
N_TICKS = 260
N_TICKS_DISPLAY = 260 - 52
N_TICKS_BURN = N_TICKS - N_TICKS_DISPLAY

In [ ]:
#read data into dataframe 
#append columns with eff and costs

def collect_data_for_simulation_runs_from_inst_eff(N_TESTS, dfs_inst, dfs_inst_std):

    test_mode = 'BaseVer02'
    data_mode = 'CA'
    #here think how to change it
    params_versions = [['eff' + str(x) + 'cost' + str(y) for y in np.linspace(cost_beg,cost_end,N_TESTS,endpoint=True)]\
                        for x in np.linspace(eff_beg,eff_end,N_TESTS,endpoint=True)]
    effs = [x for x in np.linspace(eff_beg,eff_end,N_TESTS,endpoint=True)]
    costs = [x for x in np.linspace(cost_beg,cost_end,N_TESTS,endpoint=True)]

    #names of columns in a file
    names = ['Tick', 'Location_x', 'Location_y', 'Uid', 'Price_per_watt', 'DC_size', 'Origin', 'Mart_type']


    for i_run in range(N_TESTS): 
        for j_run in range(N_TESTS):
            #read few files
            #save into database
            dfs_ = []
            #read last simulation result
            files = glob.glob('..\\..\\ABMIRISLab\\Saves\\May-14-2018\\BaseModel\\'+ test_mode + '\\' + data_mode + \
                     '\\' + params_versions[i_run][j_run] + '\\*[_pr].csv')
            
            files.sort(key=os.path.getmtime, reverse=True)

            #take top x files 
            for file in files[0 * N_SEEDS : 1 * N_SEEDS]:
                print(file)
                df_ = pd.read_csv(file, names=names, low_memory=False, na_values = ['-nan(ind)'])
                df_['eff'] = effs[i_run]
                df_['cost'] = costs[j_run]
                dfs_.append(df_)


            #create df from them    
            df_concat = pd.concat(dfs_)


            df_inst_mean = df_concat.groupby(level=0).mean()
            df_inst_std = df_concat.groupby(level=0).std()

            dfs_inst.append(dfs_)
